In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import backend as K
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np


def custom_tanh(x):
    return (K.exp(x) - K.exp(-x)) / (K.exp(x) + K.exp(-x))
def custom_relu(x):
    return K.maximum(0.0, x)
def custom_sigmoid(x):
    return 1 / (1 + K.exp(-x))


model = Sequential([ Dense(64, input_shape=(10,)), 
    Activation(custom_relu), Dense(64), Activation(custom_tanh), Dense(1), Activation(custom_sigmoid) ])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


np.random.seed(0)
X_train = np.random.random((100, 10))  
Y_train = np.random.randint(2, size=(100, 1))  
model.fit(X_train, Y_train, epochs=10, batch_size=10)
predictions = model.predict(X_train[:5])


print("Predictions:\n", predictions)
binary_predictions = (predictions > 0.5).astype(int)
accuracy = accuracy_score(Y_train[:5], binary_predictions)
print("Accuracy:", accuracy)
# Precision
precision = precision_score(Y_train[:5], binary_predictions)
print("Precision:", precision)
# Recall
recall = recall_score(Y_train[:5], binary_predictions)
print("Recall:", recall)
# F1-score
f1 = f1_score(Y_train[:5], binary_predictions)
print("F1-score:", f1)
# Confusion Matrix
conf_matrix = confusion_matrix(Y_train[:5], binary_predictions)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def train_model(optimizer):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=0)
    return history.history['accuracy'][-1] * 100
accuracy_gd = train_model('sgd')
accuracy_sgd = train_model('adam')

print("Gradient Descent Accuracy: {:.2f}%".format(accuracy_gd))
print("Stochastic Gradient Descent Accuracy: {:.2f}%".format(accuracy_sgd))

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Define user-defined activation function
def custom_activation(x):
    return tf.nn.leaky_relu(x, alpha=0.2)

# Function to create and train the model with given dropout probability or clipout value
def train_model(activation_type, value):
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),  # Flatten the input
        tf.keras.layers.Dense(64, activation=custom_activation),
        tf.keras.layers.Dense(64, activation=custom_activation),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    if activation_type == 'dropout':
        model.add(tf.keras.layers.Dropout(value))
        model.add(tf.keras.layers.Dropout(value))
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    elif activation_type == 'clipout':
        opt = tf.keras.optimizers.Adam(clipvalue=value)  # Gradient clipping
        model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=5, batch_size=64, verbose=0)
    _, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    return test_accuracy

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define a list of dropout probabilities to test
dropout_probs = [0.1, 0.2, 0.3, 0.4, 0.5]
# Define a list of clipout values to test
clipout_values = [0.1, 0.2, 0.3, 0.4, 0.5]

# Initialize lists to store accuracies
dropout_accuracies = []
clipout_accuracies = []

# Test dropout probabilities
for dropout_prob in dropout_probs:
    accuracy = train_model('dropout', dropout_prob)
    dropout_accuracies.append(accuracy)
    print("Dropout Probability:", dropout_prob, "- Test Accuracy:", accuracy)

# Test clipout values
for clipout_value in clipout_values:
    accuracy = train_model('clipout', clipout_value)
    clipout_accuracies.append(accuracy)
    print("Clipout Value:", clipout_value, "- Test Accuracy:", accuracy)

# Plotting the results
plt.figure(figsize=(10, 5))
plt.plot(dropout_probs, dropout_accuracies, marker='o', label='Dropout')
plt.plot(clipout_values, clipout_accuracies, marker='s', label='Clipout')
plt.xlabel('Probability/Value')
plt.ylabel('Test Accuracy')
plt.title('Effect of Dropout and Clipout on Test Accuracy')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

x_task1 = np.random.rand(1000, 10)
x_task2 = np.random.rand(1000, 10)  
y_task1 = np.random.rand(1000, 1) 
y_task2 = np.random.rand(1000, 1) 
 
input_layer = Input(shape=(10,))

branch_task1 = Dense(64, activation='relu')(input_layer)
output_task1 = Dense(1, name='output_task1')(branch_task1)

branch_task2 = Dense(64, activation='relu')(input_layer)
output_task2 = Dense(1, name='output_task2')(branch_task2)

model_task1 = Model(inputs=input_layer, outputs=output_task1)
model_task2 = Model(inputs=input_layer, outputs=output_task2)

model_task1.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model_task2.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

multitask_model = Model(inputs=input_layer, outputs=[output_task1, output_task2])

multitask_model.compile(optimizer='adam',
                        loss={'output_task1': 'mean_squared_error',
                              'output_task2': 'mean_squared_error'},
                        loss_weights={'output_task1': 0.5, 'output_task2': 0.5},
                        metrics={'output_task1': 'mae', 'output_task2': 'mae'})

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = multitask_model.fit(x_task1, [y_task1, y_task2], validation_split=0.2, epochs=50, callbacks=[early_stopping])

# Plotting Training and Validation Loss
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Task 1 Training Loss')
plt.plot(history.history['val_output_task1_loss'], label='Task 1 Validation Loss')
plt.plot(history.history['output_task2_loss'], label='Task 2 Training Loss')
plt.plot(history.history['val_output_task2_loss'], label='Task 2 Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Plotting Early Stopping
plt.figure(figsize=(10, 6))
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(early_stopping.stopped_epoch, history.history['val_loss'][early_stopping.stopped_epoch], 'ro', label='Stopped Epoch')
plt.title('Early Stopping')
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

max_features = 10000
maxlen = 100

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

model = Sequential([
    Embedding(max_features, 32),
    SimpleRNN(32),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=100, batch_size=128, validation_split=0.2)

loss, accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy:", accuracy)

def predict_sentiment(text, model, maxlen=maxlen):
    tokens = imdb.get_word_index()
    tokens = {k: (v + 3) for k, v in tokens.items()}
    tokens["<PAD>"] = 0
    tokens["<START>"] = 1
    tokens["<UNK>"] = 2
    tokens["<UNUSED>"] = 3
    
    reverse_tokens = dict([(value, key) for (key, value) in tokens.items()])
    
    sequence = [tokens.get(word, tokens["<UNK>"]) for word in text.split()]
    sequence = pad_sequences([sequence], maxlen=maxlen)
    
    prediction = model.predict(sequence)[0, 0]
    sentiment = "positive" if prediction >= 0.5 else "negative"
    return sentiment, prediction

#text1 = "This movie was great! I really enjoyed it."
#text2 = "The acting was terrible and the plot was boring."



text1= input(" ENTER YOUR TEXT : ")
text2= input ("ENTER SECOND TEXT : ")

sentiment1, score1 = predict_sentiment(text1, model)
sentiment2, score2 = predict_sentiment(text2, model)

print(f"Text: '{text1}' | Sentiment: {sentiment1} | Score: {score1:.4f}")
print(f"Text: '{text2}' | Sentiment: {sentiment2} | Score: {score2:.4f}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import datetime

ticker = 'AAPL'
data = yf.download(ticker, start='2010-01-01', end=datetime.datetime.now().strftime('%Y-%m-%d'))['Close']

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data.values.reshape(-1, 1))

time_step = 60
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

X_train, y_train = [], []
for i in range(time_step, len(train_data)):
    X_train.append(train_data[i-time_step:i, 0])
    y_train.append(train_data[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_test, y_test = [], []
for i in range(time_step, len(test_data)):
    X_test.append(test_data[i-time_step:i, 0])
    y_test.append(test_data[i, 0])
X_test, y_test = np.array(X_test), np.array(y_test)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_step, 1)),
    LSTM(50),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

history = model.fit(X_train, y_train, epochs=5, batch_size=1, validation_data=(X_test, y_test))

train_predict = scaler.inverse_transform(model.predict(X_train))
test_predict = scaler.inverse_transform(model.predict(X_test))

plt.figure(figsize=(16,8))
plt.plot(data.index, data.values, label='Actual Stock Price')
plt.plot(data.index[time_step:train_size], train_predict, label='Training Predictions')
plt.plot(data.index[train_size+time_step:], test_predict, label='Testing Predictions')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()
fig, ax1 = plt.subplots(figsize=(12, 6))
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.plot(history.history['loss'], label='Training Loss', color='tab:blue')
ax1.plot(history.history['val_loss'], label='Validation Loss', color='tab:orange')
ax1.tick_params(axis='y')
ax2 = ax1.twinx()
ax2.set_ylabel('MAE')
ax2.plot(history.history['mae'], label='Training MAE', color='tab:green')
ax2.plot(history.history['val_mae'], label='Validation MAE', color='tab:red')
ax2.tick_params(axis='y')
fig.tight_layout()
fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))
plt.title('Model Loss and MAE')
plt.show()

def predict_next_day(model, data, time_step):
    last_data = scaler.transform(data[-time_step:].values.reshape(-1, 1))
    next_day_prediction = model.predict(last_data.reshape(1, time_step, 1))
    return scaler.inverse_transform(next_day_prediction)[0, 0]

next_day_price = predict_next_day(model, data, time_step)
print(f'Next day predicted stock price: {next_day_price}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
import datetime

ticker = 'AAPL'
data = yf.download(ticker, start='2010-01-01', end=datetime.datetime.now().strftime('%Y-%m-%d'))['Close']
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.values.reshape(-1, 1))

time_step = 60
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

def create_sequences(data, time_step):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i-time_step:i])
        y.append(data[i])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data, time_step)
X_test, y_test = create_sequences(test_data, time_step)

model = Sequential([
    GRU(50, return_sequences=True, input_shape=(time_step, 1)),
    GRU(50),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
history = model.fit(X_train, y_train, epochs=5, batch_size=1, validation_data=(X_test, y_test))

train_predict = scaler.inverse_transform(model.predict(X_train))
test_predict = scaler.inverse_transform(model.predict(X_test))

plt.figure(figsize=(16, 8))
plt.plot(data.index, data.values, label='Actual Stock Price')
plt.plot(data.index[time_step:train_size], train_predict, label='Training Predictions')
plt.plot(data.index[train_size + time_step:], test_predict, label='Testing Predictions')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

fig, ax1 = plt.subplots(figsize=(12, 6))
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.plot(history.history['loss'], label='Training Loss', color='tab:blue')
ax1.plot(history.history['val_loss'], label='Validation Loss', color='tab:orange')
ax2 = ax1.twinx()
ax2.set_ylabel('MAE')
ax2.plot(history.history['mae'], label='Training MAE', color='tab:green')
ax2.plot(history.history['val_mae'], label='Validation MAE', color='tab:red')
fig.tight_layout()
fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))
plt.title('Model Loss and MAE')
plt.show()

def predict_next_day(model, data, time_step):
    last_data = scaler.transform(data[-time_step:].values.reshape(-1, 1))
    next_day_prediction = model.predict(last_data.reshape(1, time_step, 1))
    return scaler.inverse_transform(next_day_prediction)[0, 0]

next_day_price = predict_next_day(model, data, time_step)
print(f'Next day predicted stock price: {next_day_price}')


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt
import numpy as np

mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.plot(history.history['loss'], label='Training Loss', color='tab:blue')
ax1.plot(history.history['val_loss'], label='Validation Loss', color='tab:orange')
ax2 = ax1.twinx()
ax2.set_ylabel('Accuracy')
ax2.plot(history.history['accuracy'], label='Training Accuracy', color='tab:green')
ax2.plot(history.history['val_accuracy'], label='Validation Accuracy', color='tab:red')
fig.tight_layout()
fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))
plt.title('Model Loss and Accuracy')
plt.show()

predictions = model.predict(X_test)

predicted_labels = np.argmax(predictions, axis=1)

num_predictions = 10
print("Predictions with Images:")
plt.figure(figsize=(12, 6))
for i in range(num_predictions):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_test[i].reshape(28, 28), cmap='gray')
    plt.title(f"Predicted: {predicted_labels[i]}")
    plt.axis('off')
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import numpy as np
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train, y_train = x_train[:5000], y_train[:5000]
x_test, y_test = x_test[:1000], y_test[:1000]

x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int64)
y_test = tf.convert_to_tensor(y_test, dtype=tf.int64)

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

def plot_loss(losses, title):
    plt.plot(losses)
    plt.title(title)
    plt.xlabel("Iteration")
    plt.ylabel("Loss")
    plt.show()

def create_adversarial_pattern(model, input_image, input_label):
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = model(input_image)
        loss = tf.keras.losses.sparse_categorical_crossentropy(input_label, prediction)
    gradient = tape.gradient(loss, input_image)
    signed_grad = tf.sign(gradient)
    return signed_grad, loss

def adversarial_training(model, x_train, y_train, epochs=5, epsilon=0.1):
    batch_size = 64
    losses = []
    for epoch in range(epochs):
        epoch_loss = 0
        for i in range(0, len(x_train), batch_size):
            x_batch = x_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            perturbations, loss = create_adversarial_pattern(model, x_batch, y_batch)
            epoch_loss += np.mean(loss)
            x_adv = x_batch + epsilon * perturbations
            x_adv = tf.clip_by_value(x_adv, 0, 1)
            model.train_on_batch(x_adv, y_batch)
        print(f"Epoch {epoch+1}/{epochs} completed")
        losses.append(epoch_loss/epochs)
    plot_loss(losses, "Adversarial Training Loss")

model.fit(x_train, y_train, epochs=5, batch_size=64)
adversarial_training(model, x_train, y_train, epochs=5)
tangent_vectors = tf.random.normal((28, 28))

def tangent_propagation_loss(model, x, y, lambda_tangent=0.1):
    with tf.GradientTape() as tape:
        tape.watch(x)
        predictions = model(x)
        classification_loss = tf.keras.losses.sparse_categorical_crossentropy(y, predictions)

    gradients = tape.gradient(predictions, x)
    tangent_loss = tf.reduce_sum(tf.square(tf.tensordot(gradients, tangent_vectors, axes=1)))

    return classification_loss + lambda_tangent * tangent_loss


def train_with_tangent_propagation(model, x_train, y_train, epochs=5, lambda_tangent=0.1):
    optimizer = tf.keras.optimizers.Adam()
    batch_size = 64
    losses = []
    for epoch in range(epochs):
        epoch_loss = 0
        for i in range(0, len(x_train), batch_size):
            x_batch = x_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            with tf.GradientTape() as tape:
                loss = tangent_propagation_loss(model, x_batch, y_batch, lambda_tangent)
            epoch_loss += np.mean(loss)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        print(f"Epoch {epoch+1}/{epochs} completed")
        losses.append(epoch_loss/epochs)
    plot_loss(losses, "Tangent Propogation Loss")

train_with_tangent_propagation(model, x_train, y_train, epochs=5)

def tangent_classifier(model, x_train, y_train, x_test, tangent_vectors):
    x_train_flat = tf.reshape(x_train, (x_train.shape[0], -1))
    x_test_flat = tf.reshape(x_test, (x_test.shape[0], -1))

    distances = tf.norm(x_test_flat[:, tf.newaxis, :] - x_train_flat[tf.newaxis, :, :], axis=2)
    nearest_indices = tf.argmin(distances, axis=1)
    predictions = tf.gather(y_train, nearest_indices)
    return predictions.numpy()

y_pred = tangent_classifier(model, x_train.numpy(), y_train.numpy(), x_test[:100].numpy(), tangent_vectors.numpy())
print("Tangent Classifier Accuracy:", np.mean(y_pred == y_test[:100].numpy()))

In [ ]:
import numpy as np
import tensorflow as tf

class SimpleRBM:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.W = tf.Variable(tf.random.normal([input_size, output_size], 0.01))
        self.h_bias = tf.Variable(tf.zeros([output_size]))
        self.v_bias = tf.Variable(tf.zeros([input_size]))

    def sample(self, probs):
        return tf.nn.relu(tf.sign(probs - tf.random.uniform(tf.shape(probs))))

    def step(self, v):
        h_probs = tf.nn.sigmoid(tf.matmul(v, self.W) + self.h_bias)
        h_sample = self.sample(h_probs)
        v_probs = tf.nn.sigmoid(tf.matmul(h_sample, tf.transpose(self.W)) + self.v_bias)
        v_sample = self.sample(v_probs)
        return h_sample, v_sample

    def train(self, data, epochs=1000, lr=0.1):
        for epoch in range(epochs):
            for v in data:
                v = np.reshape(v, (1, -1))
                h_sample, v_sample = self.step(v)
                pos_grad = tf.matmul(tf.transpose(v), h_sample)
                neg_grad = tf.matmul(tf.transpose(v_sample), self.step(v_sample)[0])
                self.W.assign_add(lr * (pos_grad - neg_grad))
                self.v_bias.assign_add(lr * tf.reduce_mean(v - v_sample, axis=0))
                self.h_bias.assign_add(lr * tf.reduce_mean(h_sample - self.step(v_sample)[0], axis=0))

    def transform(self, data):
        data = np.reshape(data, (1, -1))
        h_probs = tf.nn.sigmoid(tf.matmul(data, self.W) + self.h_bias)
        return h_probs


input_size = 6
hidden_size_1 = 3
hidden_size_2 = 2
epochs = 1000
learning_rate = 0.1

data = np.array([[1, 1, 1, 0, 0, 0],
                 [1, 0, 1, 0, 0, 0],
                 [1, 1, 1, 0, 0, 0],
                 [0, 0, 1, 1, 1, 0],
                 [0, 0, 1, 1, 0, 0],
                 [0, 0, 1, 1, 1, 0]], dtype=np.float32)

rbm1 = SimpleRBM(input_size, hidden_size_1)
rbm1.train(data, epochs, learning_rate)
h1 = np.array([rbm1.transform(v) for v in data])

rbm2 = SimpleRBM(hidden_size_1, hidden_size_2)
rbm2.train(h1, epochs, learning_rate)
h2 = np.array([rbm2.transform(h) for h in h1])

print("Original Data:\n", data)
print("Features from RBM1:\n", h1)
print("Features from RBM2:\n", h2)